# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [64]:
# Run this command in the terminal to install the required packages:
# pip install -r requirements.txt

In [65]:
import os
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

# read local .env file
_ = load_dotenv(find_dotenv())

# Initialize OpenAI client
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Set the model to use
llm_model = "gpt-4o"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [66]:
def get_completion(prompt, model=llm_model):
    response = client.responses.create(
        model=model,
        input=prompt,
        temperature=0,
    )
    return response.output_text


In [67]:
get_completion("What is 1+1?")

'1 + 1 equals 2.'

In [68]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [69]:
response = get_completion(prompt)

response

"I'm quite upset that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up. I could really use your help right now."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Model

In [70]:
from langchain_openai import ChatOpenAI

In [71]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F70601E560>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F70601C6D0>, root_client=<openai.OpenAI object at 0x000001F705AD7C70>, root_async_client=<openai.AsyncOpenAI object at 0x000001F70601E3E0>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

### Prompt template

In [72]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [73]:
from langchain.prompts import ChatPromptTemplate

# Create a prompt template from a template string
prompt_template = ChatPromptTemplate.from_template(template_string)


In [74]:
# Get the prompt template
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [75]:
# Get the input variables of the prompt template
prompt_template.messages[0].prompt.input_variables

['style', 'text']

`format_messages` is a **batch operation** that:
1. Takes variable values as input
2. Finds **ALL** messages in the prompt template that contain those variables
3. Replaces the variables in ALL those messages with the same values
4. Returns a list of ALL formatted messages

In [76]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

# Format the messages using the template
customer_messages = prompt_template.format_messages(
                    style=customer_style, # Applied to ALL messages containing {style}
                    text=customer_email)  # Applied to ALL messages containing {text}

**LangChain Message Types:**
LangChain has different message types for different roles:

1. **HumanMessage**: Messages from humans/users
2. **AIMessage**: Messages from AI assistants
3. **SystemMessage**: System instructions
4. **ChatMessage**: Generic messages with custom roles

In [77]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [78]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [79]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [80]:
print(customer_response.content)

I'm quite upset that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now.


In [81]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

# Format the messages using the template
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [82]:
service_response = chat.invoke(service_messages)

print(service_response.content)

Ahoy, me hearty customer! I be regretful to inform ye that the warranty be not coverin' the cleanin' expenses for yer galley, as it seems ye may have misused yer blender by forgettin' to secure the lid afore startin' the contraption. Alas, such be the way of the seas! Fair winds to ye!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [83]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [84]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [85]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [86]:
# Format the messages using the template: replace the {text} variable with the customer_review
messages = prompt_template.format_messages(text=customer_review)

# Create a chat model with temperature 0.0 and the model set to the llm_model
chat = ChatOpenAI(temperature=0.0, model=llm_model)

# Call the LLM to review the customer's review
response = chat.invoke(messages)

# Print the response
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [87]:
type(response.content)

str

In [88]:
# You will get an error by running this line of code
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [90]:
# Import the JsonOutputParser (more robust than StructuredOutputParser)
from langchain_core.output_parsers import JsonOutputParser

In [91]:
# With JsonOutputParser, we don't need ResponseSchema objects
# Instead, we'll define the expected JSON structure in our prompt
# JsonOutputParser is more flexible and robust for handling LLM JSON output

In [92]:
# Create a JsonOutputParser - no schema needed, it's more flexible
output_parser = JsonOutputParser()

In [93]:
# Get format instructions from JsonOutputParser (simpler than StructuredOutputParser)
format_instructions = output_parser.get_format_instructions()

In [94]:
print(format_instructions)

Return a JSON object.


In [95]:
# Create a prompt template for JsonOutputParser
# We'll be more explicit about the JSON structure we want

review_template_2 = """\
For the following text, extract the following information and return it as JSON:

Extract these fields:
- gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.
- delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.
- price_value: Extract any sentences about the value or price, and output them as an array of strings.

text: {text}

{format_instructions}
"""

# Create a chat prompt template using the new template
prompt = ChatPromptTemplate.from_template(template=review_template_2)

# Format the messages using the template
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [96]:
print(messages[0].content)

For the following text, extract the following information and return it as JSON:

Extract these fields:
- gift: Was the item purchased as a gift for someone else? Answer true if yes, false if not or unknown.
- delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.
- price_value: Extract any sentences about the value or price, and output them as an array of strings.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


Return a JSON object.



In [97]:
response = chat.invoke(messages)

In [98]:
# Debug: Let's see the raw response content first
print("Raw response content:")
print(repr(response.content))
print("\n" + "="*50 + "\n")

print("Formatted response content:")
print(response.content)
print("\n" + "="*50 + "\n")

# Use JsonOutputParser - it's much more robust than StructuredOutputParser
try:
    print("JSON parsed response content:")
    output_dict = output_parser.parse(response.content)
    print("✅ Successfully parsed with JsonOutputParser!")
    print("Parsed data:")
    print(output_dict)
except Exception as e:
    print(f"❌ JsonOutputParser failed with error: {e}")
    print("Raw response content:")
    print(repr(response.content))

Raw response content:
'```json\n{\n  "gift": true,\n  "delivery_days": 2,\n  "price_value": [\n    "It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."\n  ]\n}\n```'


Formatted response content:
```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


JSON parsed response content:
✅ Successfully parsed with JsonOutputParser!
Parsed data:
{'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [99]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [100]:
type(output_dict)

dict

In [101]:
output_dict.get('delivery_days')

2

### JsonOutputParser can stream partial JSON objects

The following streaming demo will also demonstrate how the parser builds JSON objects incrementally, which is particularly beneficial for enhancing user experience in real-world applications.

In [102]:
# Bonus: JsonOutputParser supports streaming!
# Let's create a chain and see how it streams partial JSON objects
from langchain_core.runnables import RunnablePassthrough

# Create a simple chain: prompt -> model -> parser
chain = prompt | chat | output_parser

print("🔄 Streaming JSON parsing (shows partial objects as they're built):")
print("-" * 60)

# Stream the chain execution
for chunk in chain.stream({
    "text": customer_review,
    "format_instructions": format_instructions
}):
    print(f"Chunk: {chunk}")

print("-" * 60)
print("✅ Streaming complete!")


🔄 Streaming JSON parsing (shows partial objects as they're built):
------------------------------------------------------------
Chunk: {}
Chunk: {'gift': True}
Chunk: {'gift': True, 'delivery_days': 2}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': []}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ['']}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other"]}
Chunk: {'gift': True, 'delivery_days': 2, 'price_value': 

In [103]:
# 📊 Comparison: JsonOutputParser vs StructuredOutputParser
print("=" * 70)
print("📊 COMPARISON: JsonOutputParser vs StructuredOutputParser")
print("=" * 70)

print("\n✅ JsonOutputParser Advantages:")
print("   • More robust JSON parsing (handles common LLM formatting issues)")
print("   • Supports streaming partial JSON objects")
print("   • Simpler setup (no need for ResponseSchema objects)")
print("   • Better error handling for malformed JSON")
print("   • More flexible - works with any JSON structure")

print("\n❌ StructuredOutputParser Limitations:")
print("   • Strict JSON validation - fails on minor formatting issues")
print("   • No streaming support")
print("   • Requires ResponseSchema definitions")
print("   • Less forgiving of LLM output variations")
print("   • Immediately fails on JSON parsing errors")

print("\n💡 When to use each:")
print("   • JsonOutputParser: When you want robust, flexible JSON parsing")
print("   • StructuredOutputParser: When you need strict schema validation")

print("=" * 70)


📊 COMPARISON: JsonOutputParser vs StructuredOutputParser

✅ JsonOutputParser Advantages:
   • More robust JSON parsing (handles common LLM formatting issues)
   • Supports streaming partial JSON objects
   • Simpler setup (no need for ResponseSchema objects)
   • Better error handling for malformed JSON
   • More flexible - works with any JSON structure

❌ StructuredOutputParser Limitations:
   • Strict JSON validation - fails on minor formatting issues
   • No streaming support
   • Requires ResponseSchema definitions
   • Less forgiving of LLM output variations
   • Immediately fails on JSON parsing errors

💡 When to use each:
   • JsonOutputParser: When you want robust, flexible JSON parsing
   • StructuredOutputParser: When you need strict schema validation
